In [13]:
import sys  
sys.path.insert(0, '/home/yon/jupyter-server/mlg/src/')

import config
import loader
import model as m
import preprocessing

# if u want to use the big config
#config = config.big

In [14]:
# temporary imports (delete later)
from pprint import pprint
import random
import numpy as np
import time
import torch
import torch_geometric

In [15]:
device = torch.device("cuda")

In [16]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
ghetero = loader.get_ghetero()
data_train, data_val, data_test = loader.get_datasets()

Loading ghetero from pickle ...
Loading datasets from pickle ...


In [21]:
class Impostor(torch.nn.Module):
    """Intelligent Mmusic Prediction Operating System with Top-level Optimization and Recommendation."""

    def __init__(self, data_train):
        super().__init__()
        self.model = m.HeteroModel(64, ghetero.x_dict, ghetero.metadata())
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01)

        edge_label_index = data_train["track", "contains", "playlist"].edge_label_index
        edge_label = data_train["track", "contains", "playlist"].edge_label
        data_train = data_train.to(device)
        self.train_loader = torch_geometric.loader.LinkNeighborLoader(
            data=data_train,
            num_neighbors=[20, 10],
            neg_sampling_ratio=2.0,
            edge_label_index=(("track", "contains", "playlist"), edge_label_index),
            edge_label=edge_label,
            batch_size=128,
            shuffle=True,
        )

    def forward(self, data):
        return self.model.forward(data)
    
    def train(self):
        self.model.train()

        total_examples = total_loss = 0
        for batch in self.train_loader:
            self.optimizer.zero_grad()
            batch_size = 100
            out = self.model(batch)
            loss = torch.nn.functional.cross_entropy(
                out, batch["track", "contains", "playlist"].edge_label
            )
            loss.backward()
            self.optimizer.step()

            total_examples += batch_size
            print(f'Loss: {loss:.4f}')
            total_loss += float(loss) * batch_size

        return total_loss / total_examples

# create training mask for playlist nodes
train_mask = torch.zeros(ghetero["playlist"].x.shape[0], dtype=torch.bool)
train_mask[torch.randperm(train_mask.shape[0])[:int(train_mask.shape[0]*0.8)]] = True

ghetero["playlist"].train_mask = train_mask

ghetero["playlist"].y = torch.LongTensor([1]*ghetero["playlist"].x.shape[0])

model = Impostor(data_train)
model = model.to(device)
model.train()


RuntimeError: Expected a 'cuda' device type for generator but found 'cpu'